<a href="https://colab.research.google.com/github/Abimbola-ai/Blood-Spectorscopy/blob/main/Data_Preparation_Blood_Spectroscopy_(ML_Models).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas.util.testing as tm
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from prettytable import PrettyTable
import tqdm
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')
seed = 42
import pandas.util.testing as tm

/var/folders/sf/jsnvw4tn5bd5khnq5nncxv940000gn/T/ipykernel_72456/4160731515.py:4: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train = pd.read_csv("clean_train.csv",index_col=[0])
test = pd.read_csv("clean_test.csv",index_col=[0])

In [3]:
train.columns

Index(['donation_id', 'absorbance0', 'absorbance1', 'absorbance2',
       'absorbance3', 'absorbance4', 'absorbance5', 'absorbance6',
       'absorbance7', 'absorbance8',
       ...
       'hdl_cholesterol_human', 'hemoglobin(hgb)_human',
       'cholesterol_ldl_human', 'std', 'low_abs', 'high_abs', 'med_abs',
       'mean_abs', 'std_abs', 'sum_abs'],
      dtype='object', length=183)

In [4]:
test.head()

,donation_id,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,...,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,std,low_abs,high_abs,med_abs,mean_abs,std_abs,sum_abs
355,ID_1961,0.499920,0.504608,0.507929,0.519943,0.529032,0.536118,0.539690,0.541710,0.548590,...,NaN,NaN,NaN,0.0064,0.499920,1.545996,0.890386,0.984720,0.361753,167.402402
356,ID_2067,0.497758,0.498091,0.503083,0.507249,0.513739,0.521944,0.528798,0.533743,0.541470,...,NaN,NaN,NaN,0.0038,0.497758,1.561147,0.868207,0.989850,0.372864,168.274451
357,ID_2155,0.551436,0.553079,0.556722,0.560722,0.565583,0.570889,0.578668,0.585142,0.594233,...,NaN,NaN,NaN,0.0043,0.551436,1.815660,1.001062,1.138021,0.446924,193.463542
358,ID_2211,0.525926,0.528948,0.532222,0.540022,0.547279,0.554348,0.558680,0.563768,0.570031,...,NaN,NaN,NaN,0.0046,0.525926,1.778915,0.920952,1.063624,0.424112,180.816128
359,ID_2327,0.559814,0.559612,0.561561,0.565162,0.567556,0.572362,0.579762,0.587761,0.599415,...,NaN,NaN,NaN,0.0052,0.559612,1.443285,0.947634,1.013858,0.318206,172.355897


In [5]:
train.head()

,donation_id,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,...,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,std,low_abs,high_abs,med_abs,mean_abs,std_abs,sum_abs
0,1975,0.505055,0.506570,0.511544,0.516300,0.524067,0.530191,0.536138,0.542317,0.551118,...,0,1,1,0.0073,0.505055,1.927817,0.958842,1.135224,0.497048,192.988096
1,1982,0.523531,0.523485,0.526291,0.532875,0.538397,0.543180,0.547520,0.553433,0.561674,...,0,1,2,0.0043,0.523485,1.734845,0.916559,1.061186,0.421130,180.401567
2,1989,0.516449,0.519226,0.523619,0.532366,0.540438,0.547985,0.552716,0.556823,0.561827,...,0,1,2,0.0046,0.516449,1.884757,0.925547,1.087614,0.462437,184.894345
3,2018,0.514163,0.516174,0.521443,0.528857,0.534581,0.543953,0.551666,0.557929,0.565811,...,2,1,1,0.0082,0.514163,1.868637,0.942196,1.113725,0.469601,189.333202
4,2045,0.471717,0.474344,0.477808,0.483401,0.491121,0.496970,0.503694,0.509696,0.516301,...,1,1,1,0.0051,0.471717,1.917085,0.878465,1.075850,0.493414,182.894524


In [6]:
# Raw Data
y_hdl_cholesterol_human = train['hdl_cholesterol_human']
y_hemoglobin_hgb_human = train['hemoglobin(hgb)_human']
y_cholesterol_ldl_human = train["cholesterol_ldl_human"]
train_drop = train.drop(['donation_id', 'hdl_cholesterol_human', 'hemoglobin(hgb)_human',
       'cholesterol_ldl_human'], axis=1)
# test = test.drop(['hdl_cholesterol_human', 'hemoglobin(hgb)_human',
#        'cholesterol_ldl_human'], axis=1)
# test_drop = test.drop(['donation_id'], axis=1)
df_train = train_drop.copy()
print(df_train.shape)
print(y_hdl_cholesterol_human.shape)
print(y_hemoglobin_hgb_human.shape)
print(y_cholesterol_ldl_human.shape)

(355, 179)
(355,)
(355,)
(355,)


In [7]:
X_cols = list(df_train.columns)
X_test = test[X_cols]

In [8]:
def hyperparameter_model(models, params, x_train, y_train):
    '''
    Hyperparameter tuning with RepeatedStratifiedKFold follow by GridSearchCV
    
    Parameters:
    models: Instance of the model
    params: list of parameters with value fr tuning (dict)
    
    Return:
    grid_clf: return gridsearch model    
    '''
    str_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
    grid_clf = GridSearchCV(models, params, cv=str_cv, return_train_score=True, scoring='roc_auc')
    grid_clf.fit(x_train, y_train)
    return grid_clf

def forward_selection_model(x_train, y_train,model, top_n=10):
    top_column = []
    exist_score = 0
    for n in range(top_n):
        print('for {} feature'.format(n+1))
        flag = 0
        for i in tqdm.tqdm_notebook(range(test_scaled.shape[1])):
            if len(top_column) == 0:
                str_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
                score = cross_val_score(model,x_train[:,[i]],y_train,cv=str_cv,scoring='roc_auc')
                if exist_score < np.mean(score):
                    top_current = i
                    exist_score = np.mean(score)
                    flag = 1
            elif i not in top_column:
                str_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
                score = cross_val_score(model,x_train[:,np.concatenate((top_column,[i]))],y_train,cv=str_cv,scoring='roc_auc')
                if exist_score < np.mean(score):
                    top_current = i
                    exist_score = np.mean(score)
                    flag = 1
        
        if flag == 1:
            print('Current top feature {} and score: {}'.format(top_current,exist_score))
            print('Appended to top column')
            top_column.append(top_current)
            print(top_column)
        else:
            break
    return top_column

def plot_feature_importance(model, model_name, top_n = 10):
    '''  
    Parameters:
    model: model 
    model_name: name of the model
  
    Return:
    Top 10 features of the dataset  
    '''       
    column_name = train.drop(['id','target'], axis=1).columns
    if model_name == 'log_model':
        feat_imp_coef = model.coef_.ravel()
    else:
        feat_imp_coef = model.feature_importances_
    temp = pd.DataFrame(data=np.column_stack((column_name, feat_imp_coef)), columns=['col_name','coef'])
    temp = temp.sort_values(by='coef', ascending=False).reset_index()
    df = temp
    temp = temp[:top_n]
    return temp

In [9]:
reverse_mapping = {'0' : 'low', '1' : 'ok', '2' : 'high'}

#### Build each model individually

##### A. Baseline Model - Logistic Regression

- hdl cholesterol human

In [10]:
log_reg = LogisticRegression(random_state=seed)
log_reg.fit(df_train,y_hdl_cholesterol_human)


LogisticRegression(random_state=42)

In [11]:
hdl_cholesterol_human = log_reg.predict(X_test)

In [12]:
test['hdl_cholesterol_human'] = hdl_cholesterol_human

In [13]:
test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
test['hdl_cholesterol_human'].value_counts()

1    60
0     1
Name: hdl_cholesterol_human, dtype: int64

- ldl cholesterol human

In [14]:
log_reg2 = LogisticRegression(random_state=seed)
log_reg2.fit(df_train,y_cholesterol_ldl_human)

LogisticRegression(random_state=42)

In [15]:
cholesterol_ldl_human = log_reg2.predict(X_test)
test['cholesterol_ldl_human'] = cholesterol_ldl_human
test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
test['cholesterol_ldl_human'].value_counts()

1    54
2     7
Name: cholesterol_ldl_human, dtype: int64

- hemoglobin human

In [16]:
log_reg3 = LogisticRegression(random_state=seed)
log_reg3.fit(df_train,y_hemoglobin_hgb_human)

LogisticRegression(random_state=42)

In [17]:

hemoglobin_hgb_human = log_reg3.predict(X_test)
test['hemoglobin_hgb_human'] = hemoglobin_hgb_human
test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
test['hemoglobin_hgb_human'].value_counts()


1    61
Name: hemoglobin_hgb_human, dtype: int64

In [18]:
maindir = "/Users/Abimbola/Documents/MSc Computer Science/Semester 2/Computer science project/Blood-Spectorscopy/Data" # Directory with your files
ss =  maindir+"/Updated_Sample_Submission.csv"
ss = pd.read_csv(ss)
ss.head()

,Unnamed: 0,Donation_ID,target
0,0,ID_1961_cholesterol_ldl_human,ok
1,1,ID_1961_hdl_cholesterol_human,ok
2,2,ID_1961_hemoglobin(hgb)_human,ok
3,3,ID_2067_cholesterol_ldl_human,ok
4,4,ID_2067_hdl_cholesterol_human,ok


In [19]:
def transform_c_hdl(row):
    return pd.Series([str(row["donation_id"]) + "_hdl_cholesterol_human", row['hdl_cholesterol_human']])

In [20]:
sample_c_hdl = pd.DataFrame(columns = ss.columns)

In [22]:
from tqdm.notebook import tqdm

tqdm.pandas()
sample_c_hdl[['Donation_ID', 'target']] = test.progress_apply(transform_c_hdl, axis = 1)

  0%|          | 0/61 [00:00<?, ?it/s]

In [23]:
def transform_hemo(row):
    return pd.Series([str(row["donation_id"]) + "_hemoglobin(hgb)_human", row["hemoglobin(hgb)_human"]])

In [24]:
sample_hemo = pd.DataFrame(columns = ss.columns)

In [34]:
sample_hemo[['Donation_ID', 'target']] = test.progress_apply(transform_hemo, axis = 1)

  0%|          | 0/61 [00:00<?, ?it/s]

In [33]:
sample_hemo

,Unnamed: 0,Donation_ID,target
355,NaN,ID_1961_hemoglobin(hgb)_human,NaN
356,NaN,ID_2067_hemoglobin(hgb)_human,NaN
357,NaN,ID_2155_hemoglobin(hgb)_human,NaN
358,NaN,ID_2211_hemoglobin(hgb)_human,NaN
359,NaN,ID_2327_hemoglobin(hgb)_human,NaN
...,...,...,...
411,NaN,ID_6373_hemoglobin(hgb)_human,NaN
412,NaN,ID_6381_hemoglobin(hgb)_human,NaN
413,NaN,ID_6391_hemoglobin(hgb)_human,NaN
414,NaN,ID_7755_hemoglobin(hgb)_human,NaN


In [26]:
def transform_c_ldl(row):
    return pd.Series([str(row["donation_id"]) + "_cholesterol_ldl_human", row["cholesterol_ldl_human"]])

In [27]:
sample_c_ldl = pd.DataFrame(columns = ss.columns)

In [28]:
sample_c_ldl[['Donation_ID', 'target']] = test.progress_apply(transform_c_ldl, axis = 1)

  0%|          | 0/61 [00:00<?, ?it/s]

In [29]:
submission = pd.concat([sample_c_hdl, sample_hemo, sample_c_ldl]).reset_index(drop=True)

In [30]:
submission = submission.loc[:, ~submission.columns.str.contains('^Unnamed')]

In [31]:
submission.head()

,Donation_ID,target
0,ID_1961_hdl_cholesterol_human,1.0
1,ID_2067_hdl_cholesterol_human,1.0
2,ID_2155_hdl_cholesterol_human,1.0
3,ID_2211_hdl_cholesterol_human,1.0
4,ID_2327_hdl_cholesterol_human,1.0


In [32]:
submission.to_csv("./submission.csv", index=False)